In [12]:
import torch
import math
import csv


def calculate_angle(p1, p2, p3):
    # Create vectors
    v1 = p1 - p2
    v2 = p3 - p2
    
    # Calculate dot product and magnitudes
    dot_product = torch.dot(v1, v2)
    magnitude_v1 = torch.norm(v1)
    magnitude_v2 = torch.norm(v2)
    
    # Prevent division by zero
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return None  # Undefined angle
    
    # Calculate angle in radians
    angle_rad = torch.acos(dot_product / (magnitude_v1 * magnitude_v2))
    # Convert to degrees
    return math.degrees(angle_rad.item())

# Function to calculate Euclidean distance between two points (p1, p2)
def calculate_distance(p1, p2):
    return torch.norm(p1 - p2).item()


In [13]:
def is_standing(dataset, threshold=130):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Knee_position_x'], dataset['Left Knee_position_y']])
    p3 = torch.tensor([dataset['Left Ankle_position_x'], dataset['Left Ankle_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

def is_elbow_straight(dataset, threshold=160):
    p1 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p2 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    p3 = torch.tensor([dataset['Left Wrist_position_x'], dataset['Left Wrist_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

def is_elbow_ninety(dataset, threshold1= 60 , threshold2 = 120):
    p1 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p2 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    p3 = torch.tensor([dataset['Left Wrist_position_x'], dataset['Left Wrist_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold1 and angle < threshold2


def is_shoulder_air(dataset, threshold=160):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p3 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

def is_shoulder_adjoining(dataset, threshold=30):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p3 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle < threshold


In [14]:
def create_csv(results):

    # Joint labels for reference
    joint_labels = ["Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear",
                    "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow",
                    "Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
                    "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"]

    # Function to calculate angle between three points (p1, p2, p3)
    def calculate_angle(p1, p2, p3):
        # Create vectors
        v1 = p1 - p2
        v2 = p3 - p2
        
        # Calculate dot product and magnitudes
        dot_product = torch.dot(v1, v2)
        magnitude_v1 = torch.norm(v1)
        magnitude_v2 = torch.norm(v2)
        
        # Prevent division by zero
        if magnitude_v1 == 0 or magnitude_v2 == 0:
            return None  # Undefined angle
        
        # Calculate angle in radians
        angle_rad = torch.acos(dot_product / (magnitude_v1 * magnitude_v2))
        # Convert to degrees
        return math.degrees(angle_rad.item())

    # Function to calculate Euclidean distance between two points (p1, p2)
    def calculate_distance(p1, p2):
        return torch.norm(p1 - p2).item()

    # List to store calculated angles and distances for each image
    angles_per_image = []

    # Define joint triplets for angle calculations
    joint_triplets = [
        (5, 7, 9),  # Left Elbow
        (6, 8, 10), # Right Elbow
        (5, 11, 13),# Left Hip
        (6, 12, 14),# Right Hip
        (7, 5, 11), # Left Shoulder
        (8, 6, 12), # Right Shoulder
        (11, 13, 15),# Left Knee
        (12, 14, 16) # Right Knee
    ]

    # Iterate over each image in the batch
    for img_idx in range(len(results)):  # Iterate over images (batch size)
        if len(results[img_idx]) == 0:
            print(f"No keypoints detected in image {img_idx + 1}")
            continue
        if len(results[img_idx]) > 1:
            print(f"Multiple detections in image {img_idx + 1}")
            continue
        image_angles = {"Image": img_idx + 1}
        
        # Calculate angles and distances for each joint triplet
        for triplet in joint_triplets:
            p1 = results[img_idx].keypoints.xy[0][triplet[0]]  # Correctly access the joint coordinates
            p2 = results[img_idx].keypoints.xy[0][triplet[1]]
            p3 = results[img_idx].keypoints.xy[0][triplet[2]]
            
            # Make sure the coordinates are tensors and have the shape [2] (x, y)
            angle = calculate_angle(p1, p2, p3)
            image_angles[f"{joint_labels[triplet[0]]}-{joint_labels[triplet[1]]}-{joint_labels[triplet[2]]}"] = angle
        
        # Calculate and add distances for each joint pair
        for i in range(len(results[img_idx].keypoints.xy[0])):
            for j in range(i + 1, len(results[img_idx].keypoints.xy[0])):
                p1 = results[img_idx].keypoints.xy[0][i]
                p2 = results[img_idx].keypoints.xy[0][j]
                distance = calculate_distance(p1, p2)
                image_angles[f"{joint_labels[i]}-{joint_labels[j]}_distance"] = distance

        #add the positions of the joints
        for i in range(len(results[img_idx].keypoints.xy[0])):
            p1 = results[img_idx].keypoints.xy[0][i]
            y_pos = p1[1]
            # convert tensor to float
            y_pos = y_pos.item()

            x_pos = p1[0]
            # convert tensor to float
            x_pos = x_pos.item()

            image_angles[f"{joint_labels[i]}_position_y"] = y_pos
            image_angles[f"{joint_labels[i]}_position_x"] = x_pos
            

            # to detect the position of the joint in the image

        

        
        
        # Add the "Label" field to each image
        image_angles["Label"] = results[img_idx].path.split("\\")[-2]
        
        angles_per_image.append(image_angles)


    # Save the angles and distances for all images as a CSV file
    #
    csv_file_path = "joint_angles_and_distances_per_image.csv"
    #csv_file_path = "validation_dataset.csv"
    with open(csv_file_path, mode="a", newline="") as csv_file:
        # Get fieldnames (the keys of the first entry in angles_per_image)
        if len(angles_per_image) == 0:
            print("No keypoints detected in any image")
            return
        fieldnames = angles_per_image[0].keys()
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write header only if the file is empty
        if csv_file.tell() == 0:
            writer.writeheader()

        # Write the rows for each image
        writer.writerows(angles_per_image)

    print(f"Angles and distances for each image saved to {csv_file_path}")
    return csv_file_path


In [15]:
import pandas as pd
def create_train_csv(path):
    # Load csv file
    df = pd.read_csv(path)
    hand_crafted_features = pd.DataFrame()

    hand_crafted_features["is_standing"] = df.apply(is_standing, axis=1)
    
    hand_crafted_features["is_elbow_straight"] = df.apply(is_elbow_straight, axis=1)

    hand_crafted_features["is_elbow_ninety"] = df.apply(is_elbow_ninety, axis=1)

    hand_crafted_features["is_shoulder_air"] = df.apply(is_shoulder_air, axis=1)

    hand_crafted_features["is_shoulder_adjoining"] = df.apply(is_shoulder_adjoining, axis=1)


    for column in hand_crafted_features.columns:
        hand_crafted_features[column] = hand_crafted_features[column].apply(lambda x: 0.5 if x is None else 0 if x == False else 1)

    added_joints = ["Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow","Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
            "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"]

    # put these positions and distances between them to a hand_crafted_features dataframe
    for joint in added_joints:
        hand_crafted_features[f"{joint}_position_x"] = df[f"{joint}_position_x"]
        hand_crafted_features[f"{joint}_position_y"] = df[f"{joint}_position_y"]

    # put distances between each joint to the dataframe
    for i in range(len(added_joints)):
        for j in range(i + 1, len(added_joints)):
            hand_crafted_features[f"{added_joints[i]}-{added_joints[j]}_distance"] = df[f"{added_joints[i]}-{added_joints[j]}_distance"]

    hand_crafted_features["Label"] = df["Label"]
 
    # save the hand_crafted_features to a csv file
    hand_crafted_features.to_csv("hand_crafted_features_train.csv", index=False)

    return hand_crafted_features
    

In [16]:
from ultralytics import YOLO
import os
import math

# Load the model and move it to GPU
model = YOLO("yolo11x-pose.pt")
model.to('cuda')

# Function to gather all image files recursively from subfolders
def get_image_files_from_subfolders(base_folder):
    image_files = []
    for subdir, _, files in os.walk(base_folder):  # Walk through all subdirectories
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):  # Add more image extensions if needed
                image_files.append(os.path.join(subdir, filename))
    return image_files

# Get all image files recursively from the 'images' folder
image_files = get_image_files_from_subfolders('images')

# Define batch size
batch_size = 500

# Split the image files into chunks of batch_size
batches = [image_files[i:i + batch_size] for i in range(0, len(image_files), batch_size)]

# Loop through batches
for batch in batches:
    results = None
    for image_path in batch:
        print(f"Processing: {image_path}")
        results = model(image_path, imgsz = 320, conf = 0.5, device='cuda') + results if results else model(image_path, imgsz = 320, conf = 0.5, device='cuda')

    
    # After processing the batch, create the CSV
    create_csv(results)



Processing: images\bench press\0.00144479.jpg

image 1/1 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\images\bench press\0.00144479.jpg: 320x192 1 person, 67.6ms
Speed: 2.0ms preprocess, 67.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 192)
Processing: images\bench press\0.00197119.jpg

image 1/1 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\images\bench press\0.00197119.jpg: 320x192 1 person, 27.6ms
Speed: 0.0ms preprocess, 27.6ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 192)
Processing: images\bench press\0.00226678.jpg

image 1/1 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\images\bench press\0.00226678.jpg: 320x192 1 person, 28.3ms
Speed: 1.5ms preprocess, 28.3ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 192)
Processing: images\bench press\0.00319480.jpg

image 1/1 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\images\be

In [17]:
# create train csv
path = "joint_angles_and_distances_per_image.csv"
create_train_csv(path)

,is_standing,is_elbow_straight,is_elbow_ninety,is_shoulder_air,is_shoulder_adjoining,Left Shoulder_position_x,Left Shoulder_position_y,Right Shoulder_position_x,Right Shoulder_position_y,Left Elbow_position_x,...,Right Hip-Left Knee_distance,Right Hip-Right Knee_distance,Right Hip-Left Ankle_distance,Right Hip-Right Ankle_distance,Left Knee-Right Knee_distance,Left Knee-Left Ankle_distance,Left Knee-Right Ankle_distance,Right Knee-Left Ankle_distance,Right Knee-Right Ankle_distance,Left Ankle-Right Ankle_distance
0,0.0,0.0,0.0,0.0,0.0,665.508057,804.950684,527.564636,722.201660,689.924194,...,311.128052,295.692230,458.102997,366.550507,208.989395,240.535431,60.458260,449.424255,258.109863,200.321472
1,0.5,0.0,1.0,1.0,0.0,630.867432,956.674438,595.199951,844.819885,828.369507,...,1006.176758,1006.176758,1006.176758,1006.176758,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,554.233704,...,141.627365,228.502716,330.362244,388.544006,86.897583,279.853516,314.732910,283.994385,297.558807,76.299881
3,0.0,0.0,0.0,0.0,0.0,737.802368,988.001831,511.211731,1103.709229,918.739929,...,364.505219,261.809296,423.711060,373.981567,413.173767,250.807648,478.134064,300.248840,112.172729,305.913971
4,0.0,1.0,0.0,0.0,0.0,677.520142,943.057007,680.474487,889.215454,752.390381,...,163.355087,375.177582,218.661377,516.164185,218.090225,157.775238,393.502533,229.188934,224.437302,304.515991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9376,1.0,1.0,0.0,0.0,0.0,836.981201,345.876099,766.615356,293.713867,818.000549,...,178.136368,202.232803,320.747284,356.852936,35.025677,160.341660,191.269135,126.491684,158.939270,38.181187
9377,1.0,1.0,0.0,0.0,0.0,830.727661,325.790314,759.067505,275.391632,821.900269,...,177.345734,196.974762,319.167572,349.451843,30.565998,154.921524,183.476273,126.947289,156.278412,30.366821
9378,1.0,1.0,0.0,0.0,0.0,813.515503,304.464783,746.102966,266.258667,811.188599,...,174.181412,194.851883,320.157013,349.841156,25.358063,154.551758,183.699799,129.797791,159.140961,29.698742
9379,1.0,1.0,0.0,0.0,0.0,810.096375,301.508148,744.949890,267.294800,807.244263,...,177.512726,197.983994,322.946136,352.617615,23.747805,153.935989,182.882629,130.355820,159.433777,29.671545


In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))  # Optional: Display GPU name

True
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [4]:
def is_standing(dataset, threshold=130):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Knee_position_x'], dataset['Left Knee_position_y']])
    p3 = torch.tensor([dataset['Left Ankle_position_x'], dataset['Left Ankle_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

def is_elbow_straight(dataset, threshold=160):
    p1 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p2 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    p3 = torch.tensor([dataset['Left Wrist_position_x'], dataset['Left Wrist_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

def is_elbow_ninety(dataset, threshold1= 60 , threshold2 = 120):
    p1 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p2 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    p3 = torch.tensor([dataset['Left Wrist_position_x'], dataset['Left Wrist_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold1 and angle < threshold2


def is_shoulder_air(dataset, threshold=160):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p3 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

def is_shoulder_adjoining(dataset, threshold=30):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p3 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle < threshold


In [5]:
from ultralytics import YOLO
import torch
import math
import csv
import pandas as pd
import joblib
import pandas as pd

def preprocess_image(image_path):

    model = YOLO("yolo11x-pose.pt")

    model.to('cuda')        
    results = model.predict(image_path, imgsz=320, conf=0.5)

    # List to store calculated angles and distances for each image
    angles_per_image = []

    # Define joint triplets for angle calculations
    joint_triplets = [
        (5, 7, 9),  # Left Elbow
        (6, 8, 10), # Right Elbow
        (5, 11, 13),# Left Hip
        (6, 12, 14),# Right Hip
        (7, 5, 11), # Left Shoulder
        (8, 6, 12), # Right Shoulder
        (11, 13, 15),# Left Knee
        (12, 14, 16) # Right Knee
    ]

    # Joint labels for reference
    joint_labels = ["Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear",
                "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow",
                "Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
                "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"]
    # Iterate over each image in the batch
    for img_idx in range(len(results)):  # Iterate over images (batch size)
        if len(results[img_idx]) == 0:
            print(f"No keypoints detected in image {img_idx + 1}")
            continue
        if len(results[img_idx]) > 1:
            print(f"Multiple detections in image {img_idx + 1}")
            continue
        image_angles = {"Image": img_idx + 1}
        
        # Calculate angles and distances for each joint triplet
        for triplet in joint_triplets:
            p1 = results[img_idx].keypoints.xy[0][triplet[0]]  # Correctly access the joint coordinates
            p2 = results[img_idx].keypoints.xy[0][triplet[1]]
            p3 = results[img_idx].keypoints.xy[0][triplet[2]]
            
            # Make sure the coordinates are tensors and have the shape [2] (x, y)
            angle = calculate_angle(p1, p2, p3)
            image_angles[f"{joint_labels[triplet[0]]}-{joint_labels[triplet[1]]}-{joint_labels[triplet[2]]}"] = angle
        
        # Calculate and add distances for each joint pair
        for i in range(len(results[img_idx].keypoints.xy[0])):
            for j in range(i + 1, len(results[img_idx].keypoints.xy[0])):
                p1 = results[img_idx].keypoints.xy[0][i]
                p2 = results[img_idx].keypoints.xy[0][j]
                distance = calculate_distance(p1, p2)
                image_angles[f"{joint_labels[i]}-{joint_labels[j]}_distance"] = distance

        #add the positions of the joints
        for i in range(len(results[img_idx].keypoints.xy[0])):
            p1 = results[img_idx].keypoints.xy[0][i]
            y_pos = p1[1]
            # convert tensor to float
            y_pos = y_pos.item()

            x_pos = p1[0]
            # convert tensor to float
            x_pos = x_pos.item()

            image_angles[f"{joint_labels[i]}_position_y"] = y_pos
            image_angles[f"{joint_labels[i]}_position_x"] = x_pos
            
        
        angles_per_image.append(image_angles)

        # return angles_per_image as a dataframe
        df = pd.DataFrame(angles_per_image)
        dropped_columns = ['Left Shoulder-Left Elbow-Left Wrist',
       'Right Shoulder-Right Elbow-Right Wrist',
       'Left Shoulder-Left Hip-Left Knee',
       'Right Shoulder-Right Hip-Right Knee',
       'Left Elbow-Left Shoulder-Left Hip',
       'Right Elbow-Right Shoulder-Right Hip', 'Left Hip-Left Knee-Left Ankle',
       'Right Hip-Right Knee-Right Ankle', 'Image']
        df = df.drop(columns = dropped_columns)


        hand_crafted_features = pd.DataFrame()

        hand_crafted_features["is_standing"] = df.apply(is_standing, axis=1)
        
        hand_crafted_features["is_elbow_straight"] = df.apply(is_elbow_straight, axis=1)

        hand_crafted_features["is_elbow_ninety"] = df.apply(is_elbow_ninety, axis=1)

        hand_crafted_features["is_shoulder_air"] = df.apply(is_shoulder_air, axis=1)

        hand_crafted_features["is_shoulder_adjoining"] = df.apply(is_shoulder_adjoining, axis=1)


        for column in hand_crafted_features.columns:
            hand_crafted_features[column] = hand_crafted_features[column].apply(lambda x: 0.5 if x is None else 0 if x == False else 1)

        added_joints = ["Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow","Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
                "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"]

        # put these positions and distances between them to a hand_crafted_features dataframe
        for joint in added_joints:
            hand_crafted_features[f"{joint}_position_x"] = df[f"{joint}_position_x"]
            hand_crafted_features[f"{joint}_position_y"] = df[f"{joint}_position_y"]

        # put distances between each joint to the dataframe
        for i in range(len(added_joints)):
            for j in range(i + 1, len(added_joints)):
                hand_crafted_features[f"{added_joints[i]}-{added_joints[j]}_distance"] = df[f"{added_joints[i]}-{added_joints[j]}_distance"]

        return hand_crafted_features
    

In [6]:
def predict(file_path):
    # Load the model
    model = joblib.load("models/mlp_model.joblib")
    encoder = joblib.load("models/label_encoder.joblib")

    # Load the image
    image_path = file_path
    image_angles = preprocess_image(image_path)

    # Make predictions
    predictions = model.predict(image_angles)
    # decode the predictions with using the encoder
    predictions = encoder.inverse_transform(predictions)
    return predictions


### feature denemeleri

In [64]:
# remove the xy coordinates from the dataframe
import pandas as pd

df_train = pd.read_csv("joint_angles_and_distances_per_image_balanced.csv")
df_val = pd.read_csv("validation_dataset.csv")

joint_labels = ["Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear",
                "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow",
                "Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
                "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"]
print(df_train.shape)
print(df_val.shape)


# if the column name contains the word position like that f"{joint_labels[i]}_position_x"
# then drop the column
for i in range(len(joint_labels)):
    df_train = df_train.drop(columns = [f"{joint_labels[i]}_position_x", f"{joint_labels[i]}_position_y"])
    df_val = df_val.drop(columns = [f"{joint_labels[i]}_position_x", f"{joint_labels[i]}_position_y"])

(11455, 138)
(5831, 180)


KeyError: "['Nose_position_x', 'Nose_position_y'] not found in axis"

In [10]:
dropped_columns = ['Left Shoulder-Left Elbow-Left Wrist',
       'Right Shoulder-Right Elbow-Right Wrist',
       'Left Shoulder-Left Hip-Left Knee',
       'Right Shoulder-Right Hip-Right Knee',
       'Left Elbow-Left Shoulder-Left Hip',
       'Right Elbow-Right Shoulder-Right Hip', 'Left Hip-Left Knee-Left Ankle',
       'Right Hip-Right Knee-Right Ankle', 'Image']
df_val = df_val.drop(columns = dropped_columns)

In [11]:
df_train.shape

(11455, 138)

In [12]:
df_val.shape

(5831, 137)

In [ ]:
# check if the person is standing or sitting

In [13]:
df_train.to_csv("joint_angles_and_distances_per_image_balanced_removed_xy.csv", index = False)

In [ ]:
df_val.to_csv("validation_dataset_removed_xy.csv", index = False)

-----------

# a

In [61]:
import torch 

In [62]:
#hand_crafted_features = pd.read_csv("validation_dataset_balanced.csv")
hand_crafted_features = pd.read_csv("joint_angles_and_distances_per_image.csv")

In [63]:
hand_crafted_features.shape

(11455, 172)

In [64]:
hand_crafted_features["Label"].value_counts()

Label
bench press     2291
hammer curl     2291
lat pulldown    2291
plank           2291
push up         2291
Name: count, dtype: int64

In [65]:
hand_crafted_features.drop("Image" , axis = 1, inplace = True)

In [66]:
hand_crafted_features.head()

,Nose-Left Eye_distance,Nose-Right Eye_distance,Nose-Left Ear_distance,Nose-Right Ear_distance,Nose-Left Shoulder_distance,Nose-Right Shoulder_distance,Nose-Left Elbow_distance,Nose-Right Elbow_distance,Nose-Left Wrist_distance,Nose-Right Wrist_distance,...,Right Hip_position_x,Left Knee_position_y,Left Knee_position_x,Right Knee_position_y,Right Knee_position_x,Left Ankle_position_y,Left Ankle_position_x,Right Ankle_position_y,Right Ankle_position_x,Label
0,12.476423,32.941528,1060.568970,104.924133,194.045792,210.504044,343.160065,290.698456,294.530273,213.346024,...,715.714417,1381.440796,1080.000000,1607.054443,739.180786,1619.709229,1077.896484,1712.911865,769.4646,bench press
1,45.579567,16.659624,138.062668,789.113647,238.754913,204.484955,789.113647,789.113647,789.113647,789.113647,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,bench press
2,0.000000,0.000000,0.000000,0.000000,392.825195,329.930786,394.707245,270.524017,359.846985,211.774887,...,281.136047,240.432083,469.397736,223.355743,349.675140,0.000000,0.000000,0.000000,0.0000,bench press
3,45.474056,23.322979,128.468124,875.496338,185.837234,74.269760,344.028046,122.538155,252.904373,91.975372,...,95.073547,742.361389,55.407043,718.409668,77.764114,0.000000,0.000000,0.000000,0.0000,bench press
4,0.000000,0.000000,0.000000,0.000000,1218.988403,1079.262329,1393.242188,1015.033691,1173.195312,956.925049,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,bench press


In [67]:
hand_crafted_features_copy = hand_crafted_features.copy()

In [68]:
hand_crafted_features = pd.DataFrame(hand_crafted_features_copy["Label"])

In [69]:
hand_crafted_features.head()

,Label
0,bench press
1,bench press
2,bench press
3,bench press
4,bench press


In [70]:
def is_body_flatten(dataset, threshold=140):
    p1 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p2 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p3 = torch.tensor([dataset['Left Ankle_position_x'], dataset['Left Ankle_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

In [71]:
is_body_flatten = hand_crafted_features_copy.apply(is_body_flatten, axis=1)

In [72]:
hand_crafted_features["is_body_flatten"] = is_body_flatten

In [73]:
hand_crafted_features.head()

,Label,is_body_flatten
0,bench press,True
1,bench press,None
2,bench press,False
3,bench press,False
4,bench press,None


In [74]:

def is_standing(dataset, threshold=130):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Knee_position_x'], dataset['Left Knee_position_y']])
    p3 = torch.tensor([dataset['Left Ankle_position_x'], dataset['Left Ankle_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold


In [75]:
# find is standing or sitting for each row
is_standing = hand_crafted_features_copy.apply(is_standing, axis=1)

In [76]:
is_standing.value_counts()

False    5266
True     3354
Name: count, dtype: int64

In [77]:
hand_crafted_features["is_standing"] = is_standing

In [78]:
def is_elbow_straight(dataset, threshold=160):
    p1 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p2 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    p3 = torch.tensor([dataset['Left Wrist_position_x'], dataset['Left Wrist_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

In [79]:
is_elbow_straight = hand_crafted_features_copy.apply(is_elbow_straight, axis=1)

In [80]:
hand_crafted_features["is_elbow_straight"] = is_elbow_straight

In [81]:
def is_elbow_ninety(dataset, threshold1= 60 , threshold2 = 120):
    p1 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p2 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    p3 = torch.tensor([dataset['Left Wrist_position_x'], dataset['Left Wrist_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold1 and angle < threshold2

In [82]:
is_elbow_ninety = hand_crafted_features_copy.apply(is_elbow_ninety, axis=1)

In [83]:
hand_crafted_features["is_elbow_ninety"] = is_elbow_ninety

In [84]:
def is_shoulder_air(dataset, threshold=160):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p3 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

In [85]:
is_shoulder_air = hand_crafted_features_copy.apply(is_shoulder_air, axis=1) 

In [86]:
hand_crafted_features["is_shoulder_air"] = is_shoulder_air

In [87]:
def is_shoulder_adjoining(dataset, threshold=30):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p3 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle < threshold

In [88]:
is_shoulder_adjoining = hand_crafted_features_copy.apply(is_shoulder_adjoining, axis=1)

In [89]:
hand_crafted_features["is_shoulder_adjoining"] = is_shoulder_adjoining

In [90]:
hand_crafted_features.head()

,Label,is_body_flatten,is_standing,is_elbow_straight,is_elbow_ninety,is_shoulder_air,is_shoulder_adjoining
0,bench press,True,False,False,True,False,False
1,bench press,None,None,None,None,False,True
2,bench press,False,False,False,False,False,False
3,bench press,False,True,False,True,False,True
4,bench press,None,None,False,False,True,False


In [91]:
hand_crafted_features.isnull().sum()

Label                       0
is_body_flatten          1102
is_standing              2835
is_elbow_straight        1202
is_elbow_ninety          1202
is_shoulder_air           599
is_shoulder_adjoining     599
dtype: int64

In [92]:
hand_crafted_features.shape

(11455, 7)

In [93]:
# group by according to the label and value counts for each label for is_standing
hand_crafted_features.groupby(["Label"])["is_standing"].value_counts()

Label         is_standing
bench press   False          1297
              True            336
hammer curl   False          1451
              True            572
lat pulldown  False          1272
              True            179
plank         True           1084
              False           805
push up       True           1183
              False           441
Name: count, dtype: int64

In [94]:
hand_crafted_features.groupby(["Label"])["is_elbow_straight"].value_counts()

Label         is_elbow_straight
bench press   False                1775
              True                  353
hammer curl   False                1770
              True                  520
lat pulldown  False                1552
              True                  589
plank         False                1651
              True                   40
push up       False                1449
              True                  554
Name: count, dtype: int64

In [95]:
hand_crafted_features.groupby(["Label"])["is_elbow_ninety"].value_counts()

Label         is_elbow_ninety
bench press   False              1536
              True                592
hammer curl   False              1726
              True                564
lat pulldown  False              1704
              True                437
plank         False               866
              True                825
push up       False              1452
              True                551
Name: count, dtype: int64

In [96]:
hand_crafted_features.groupby(["Label"])["is_shoulder_air"].value_counts()

Label         is_shoulder_air
bench press   False              2032
              True                229
hammer curl   False              2291
lat pulldown  False              2073
              True                133
plank         False              2025
              True                 10
push up       False              2032
              True                 31
Name: count, dtype: int64

In [97]:
hand_crafted_features.groupby(["Label"])["is_shoulder_adjoining"].value_counts()

Label         is_shoulder_adjoining
bench press   False                    1716
              True                      545
hammer curl   True                     2041
              False                     250
lat pulldown  False                    1815
              True                      391
plank         False                    1980
              True                       55
push up       False                    1408
              True                      655
Name: count, dtype: int64

In [98]:
hand_crafted_features.head()

,Label,is_body_flatten,is_standing,is_elbow_straight,is_elbow_ninety,is_shoulder_air,is_shoulder_adjoining
0,bench press,True,False,False,True,False,False
1,bench press,None,None,None,None,False,True
2,bench press,False,False,False,False,False,False
3,bench press,False,True,False,True,False,True
4,bench press,None,None,False,False,True,False


In [99]:
# make the none values to 0.5 , false 0 , true 1
for column in hand_crafted_features.columns[1:]:
    hand_crafted_features[column] = hand_crafted_features[column].apply(lambda x: 0.5 if x is None else 0 if x == False else 1)
hand_crafted_features.head()

,Label,is_body_flatten,is_standing,is_elbow_straight,is_elbow_ninety,is_shoulder_air,is_shoulder_adjoining
0,bench press,1.0,0.0,0.0,1.0,0.0,0.0
1,bench press,0.5,0.5,0.5,0.5,0.0,1.0
2,bench press,0.0,0.0,0.0,0.0,0.0,0.0
3,bench press,0.0,1.0,0.0,1.0,0.0,1.0
4,bench press,0.5,0.5,0.0,0.0,1.0,0.0


In [100]:
hand_crafted_features.groupby(["Label"])["is_shoulder_adjoining"].value_counts()

Label         is_shoulder_adjoining
bench press   0.0                      1716
              1.0                       545
              0.5                        30
hammer curl   1.0                      2041
              0.0                       250
lat pulldown  0.0                      1815
              1.0                       391
              0.5                        85
plank         0.0                      1980
              0.5                       256
              1.0                        55
push up       0.0                      1408
              1.0                       655
              0.5                       228
Name: count, dtype: int64

In [101]:
added_joints = ["Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow","Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
                "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"]

# put these positions and distances between them to a hand_crafted_features dataframe
for joint in added_joints:
    hand_crafted_features[f"{joint}_position_x"] = hand_crafted_features_copy[f"{joint}_position_x"]
    hand_crafted_features[f"{joint}_position_y"] = hand_crafted_features_copy[f"{joint}_position_y"]

hand_crafted_features.head()

,Label,is_body_flatten,is_standing,is_elbow_straight,is_elbow_ninety,is_shoulder_air,is_shoulder_adjoining,Left Shoulder_position_x,Left Shoulder_position_y,Right Shoulder_position_x,...,Right Hip_position_x,Right Hip_position_y,Left Knee_position_x,Left Knee_position_y,Right Knee_position_x,Right Knee_position_y,Left Ankle_position_x,Left Ankle_position_y,Right Ankle_position_x,Right Ankle_position_y
0,bench press,1.0,0.0,0.0,1.0,0.0,0.0,745.654358,981.648438,496.644653,...,715.714417,1335.359619,1080.000000,1381.440796,739.180786,1607.054443,1077.896484,1619.709229,769.4646,1712.911865
1,bench press,0.5,0.5,0.5,0.5,0.0,1.0,259.948273,866.272583,297.084381,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,bench press,0.0,0.0,0.0,0.0,0.0,0.0,341.592285,193.975067,266.394379,...,281.136047,223.910843,469.397736,240.432083,349.675140,223.355743,0.000000,0.000000,0.0000,0.000000
3,bench press,0.0,1.0,0.0,1.0,0.0,1.0,386.796204,847.447144,458.530121,...,95.073547,746.651733,55.407043,742.361389,77.764114,718.409668,0.000000,0.000000,0.0000,0.000000
4,bench press,0.5,0.5,0.0,0.0,1.0,0.0,686.637573,1007.204834,578.797119,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000


In [102]:
# put distances between each joint to the dataframe
for i in range(len(added_joints)):
    for j in range(i + 1, len(added_joints)):
        hand_crafted_features[f"{added_joints[i]}-{added_joints[j]}_distance"] = hand_crafted_features_copy[f"{added_joints[i]}-{added_joints[j]}_distance"]

In [103]:
hand_crafted_features.head()

,Label,is_body_flatten,is_standing,is_elbow_straight,is_elbow_ninety,is_shoulder_air,is_shoulder_adjoining,Left Shoulder_position_x,Left Shoulder_position_y,Right Shoulder_position_x,...,Right Hip-Left Knee_distance,Right Hip-Right Knee_distance,Right Hip-Left Ankle_distance,Right Hip-Right Ankle_distance,Left Knee-Right Knee_distance,Left Knee-Left Ankle_distance,Left Knee-Right Ankle_distance,Right Knee-Left Ankle_distance,Right Knee-Right Ankle_distance,Left Ankle-Right Ankle_distance
0,bench press,1.0,0.0,0.0,1.0,0.0,0.0,745.654358,981.648438,496.644653,...,367.188568,272.706329,460.467743,381.359131,408.728821,238.277725,454.208435,338.952026,110.104057,322.20639
1,bench press,0.5,0.5,0.5,0.5,0.0,1.0,259.948273,866.272583,297.084381,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,bench press,0.0,0.0,0.0,0.0,0.0,0.0,341.592285,193.975067,266.394379,...,188.985229,68.541344,359.407227,359.407227,120.934288,527.391541,527.391541,414.922272,414.922272,0.00000
3,bench press,0.0,1.0,0.0,1.0,0.0,1.0,386.796204,847.447144,458.530121,...,39.897854,33.124474,752.680420,752.680420,32.764668,744.426208,744.426208,722.606201,722.606201,0.00000
4,bench press,0.5,0.5,0.0,0.0,1.0,0.0,686.637573,1007.204834,578.797119,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [104]:
hand_crafted_features.shape

(11455, 97)

In [105]:
# save for further use
hand_crafted_features.to_csv("hand_crafted_features_train.csv", index = False)